In [ ]:
## Written by Jesse Day, Nov 12 2016
## first attempt at using the NBA/stats API to obtain data of interest.

In [1]:
import requests
import json
import pandas

In [2]:
nbastats_url = 'http://stats.nba.com/stats/'
boxscore_url = nbastats_url + 'boxscoretraditionalv2'
playbyplay_url = nbastats_url + 'playbyplayv2'

In [3]:
## first goal - load all regular-season box scores for 2015-2016 season.
## step 1: CREATE LIST OF GAME IDs

#thank you to https://github.com/gmf05/nba/blob/master/README.md for the following info:

#The NBA's Game ID, 0021400001, is a 10-digit code: XXXYYGGGGG, where XXX refers to a season prefix, YY is the season year (e.g. 14 for 2014-15), and GGGGG refers to the game number (1-1230 for a full 30-team regular season).

#Season prefixes are...
#001 : Pre Season
#002 : Regular Season
#003 : All-Star
#004 : Post Season

##if I want to adapt this to be able to account for other seasons, simply cycle through different values for season
##slight issue is that number of teams is not constant in time/teams didn't always play 82 games, but tractable
season_prefix = '002'
season = '16' #2015-2016
game = range(1,1231) #there are 82*30/2 regular-season games, or 1230 games

#"{0:0=5d}".format command below handily indicates that all numbers should be shown with five figures
gameids = [ season_prefix + season + "{0:0=5d}".format(gm) for gm in game]

In [4]:
## STEP 2 - setup entire query
# API queries reply certain parameters in requests. 
# More info available at https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation

## I'm not sure yet what all of the variables mean. Time to experiment!

## update: API seems powerful - may actually be able to separately query different times of the game
#(say, end of quarter). may mean that I never have to look at the playbyplay logs.

## Required parameters:
# GameID - see above
# StartPeriod -
# EndPeriod -
# StartRange -
# EndRange - refers to time of game in tenths of a second (28800 is end of non-overtime game)
# for instance, check out overtime game (http://stats.nba.com/game/#!/0021600121/) - endrange=31800
# RangeType -

query = {}
query['EndPeriod']=10
query['EndRange']=28800
query['RangeType']=0
query['Season']="2016-17"
query['SeasonType']="Regular+Season"
query['StartPeriod'] = 1
query['StartRange']=0
#Season=2016-17
#SeasonType=Regular+Season

In [5]:
## important discovery - the parameters have to be requested in alphabetical order
def construct_query(parameters):
    query = '?'
    
    for i, (param, value) in enumerate(sorted(parameters.items())):
        
        if i is not 0:
            query += '&'
        
        add_param = param + '=' + str(value)
        query += add_param
                     
    return(query)

In [17]:
for gameid in gameids[:10]: #cycle through list of game IDs and query each one
    query['GameID'] = gameid 
    formatted_query = construct_query(query)    
    full_query_url = boxscore_url + formatted_query
    print(full_query_url)
    
    # request the URL
    response = requests.get(full_query_url, timeout = 10) #wait 10 seconds for response
    response.raise_for_status() # raise exception if invalid response
    print(response)


http://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021600001&RangeType=0&Season=2016-17&SeasonType=Regular+Season&StartPeriod=1&StartRange=0


HTTPError: 400 Client Error: Bad Request for url: http://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021600001&RangeType=0&Season=2016-17&SeasonType=Regular+Season&StartPeriod=1&StartRange=0

In [11]:
full_query_url == test_url

True

In [10]:
test_url = "http://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021600001&RangeType=0&Season=2016-17&SeasonType=Regular+Season&StartPeriod=1&StartRange=0"

In [8]:
r2 = requests.get(test_url)
r2.raise_for_status() # raise exception if invalid response
print(r2)

HTTPError: 400 Client Error: Bad Request for url: http://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021600001&RangeType=0&Season=2016-17&SeasonType=Regular+Season&StartPeriod=1&StartRange=0

In [12]:
# request the URL and parse the JSON
response = requests.get(test_url)
response.raise_for_status() # raise exception if invalid response
print(response)

#Create Dict based on JSON response
#response = requests.get(boxscore_url)
#print(response)
gamelogs = response.json()
data = json.loads(response.text)

<Response [200]>


In [13]:
print(gamelogs['parameters'])

{'GameID': '0021600110', 'RangeType': 0, 'EndRange': 31800, 'EndPeriod': 10, 'StartPeriod': 1, 'StartRange': 0}


In [25]:
## sweet requests tutorial

response = requests.get("http://api.open-notify.org/astros.json")
data = response.json()

# 9 people are currently in space.
print(data["number"])
print(data)


5
{'number': 5, 'message': 'success', 'people': [{'name': 'Jing Haipeng', 'craft': 'Shenzhou 11'}, {'name': 'Chen Dong', 'craft': 'Shenzhou 11'}, {'name': 'Sergey Rizhikov', 'craft': 'ISS'}, {'name': 'Andrey Borisenko', 'craft': 'ISS'}, {'name': 'Shane Kimbrough', 'craft': 'ISS'}]}
